In [1]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [2]:
# Initialize API for data upload

cb_uploader = cb.AgeFileUploader(cb_config)
cb_meta = cb.Metadata(cb_config)

files_paths = [
    "./data/emas-20190411T232808.log",
    "./data/emas-20190411T234810.log",
    "./data/emas-20190412T000813.log",
    "./data/emas-20190412T120536.log",
]

files_details = cb.AgeUploadDetails(
    headers_keys={"[WH]": "[W]", "[SH]": "[S]", "[BH]": "[B]"}
)

experiment_name = 'Example AgE experiment'

uploaded_computations = []
for fp in files_paths:
    computation = cb_uploader.upload_file(file_path=fp, experiment_name=experiment_name, details=files_details)
    uploaded_computations.append(computation)
    
uploaded_configurations = list(cb_meta.experiment_configuration_api().find_by_id(id) for id in set([c.configuration_id for c in uploaded_computations]))

In [3]:
# Initialize ComputationSeries API
cb_computation_series_api = cb.ComputationSeries(cb_config)

In [4]:
# Config & Utility methods for easier plots creation

from cloudberry.converters import SeriesConverter
import cloudberry.plots as cp

props = cp.PlotProperties.default()
props.title = "Result"
props.x_axis_name = "Time [ms]"
props.y_axis_name = "Value"
props.default_series_kind = cp.PlotSeriesKind.LINE

field = 'BEST_SOLUTION_SO_FAR'
def to_plot_pack(data_pack):
    return SeriesConverter.data_series_pack_to_plot_series_pack(
        ds=data_pack,
        x_field="_time",
        y_field=field,
        y_err_field="stddev"
    )

def plot_builder(data_pack):
    plot_pack = to_plot_pack(data_pack)
    builder = cp.PlotBuilder(props)
    builder.add_pack(plot_pack)
    return builder

In [24]:
configuration0 = uploaded_configurations[0]
configuration1 = uploaded_configurations[1]

data_pack = cb_computation_series_api.for_configuration(configuration0, field)
plot_builder(data_pack).plot()

In [6]:
# Best N Computation Series API

n = 2
goal = cb.OptimizationGoal.MAX # MIN/MAX
kind = cb.OptimizationKind.FINAL_VALUE # try AREA_UNDER_CURVE

data_pack = cb_computation_series_api.best_n(n, field, goal, kind)
plot_builder(data_pack).plot()

In [23]:
data_pack = cb_computation_series_api.best_n_for_configuration(n, field, configuration0, goal, kind)
plot_builder(data_pack).plot()

In [8]:
# Thresholds Computation Series API

upper_threshold = 6.63
criteria = cb.CriteriaMode.ANY # ANY | FINAL | AVERAGE
thresholds = cb.Thresholds(upper=upper_threshold, lower=None)

data_pack = cb_computation_series_api.exceeding_thresholds(field, criteria, thresholds)
builder = plot_builder(data_pack)

# Additionaly draw threshold(s) on the plot
builder.add_trend('UPPER BOUND', cp.PlotlyTrendLine('average', cp.PlotlyTrendLineKind.CONST, upper_threshold))
builder.plot()

In [22]:
thresholds = cb.Thresholds(upper=6.7, lower=None)
data_pack = cb_computation_series_api.exceeding_thresholds_for_configuration(field, configuration0, criteria, thresholds)
plot_builder(data_pack).plot()

In [27]:
# Relative Thresholds Computation Series API

criteria = cb.CriteriaMode.ANY
thresholds = cb.Thresholds(upper=1, lower=None) # exceeding by 1% more relatively to average series 
thresholds_type = cb.ThresholdsType.PERCENTS

data_pack = cb_computation_series_api.exceeding_thresholds_relatively(field, configuration0, criteria, thresholds, thresholds_type)
plot_builder(data_pack).plot()

In [11]:
# Initialize ConfigurationSeries API
cb_configuration_series_api = cb.ConfigurationSeries(cb_config)

In [12]:
data_pack = cb_configuration_series_api.for_configurations(uploaded_configurations, field)
plot_builder(data_pack).plot()

In [13]:
data_pack = cb_configuration_series_api.for_experiment(experiment_name, field)
plot_builder(data_pack).plot()

In [14]:
# Best N Configuration Series API

n = 2
goal = cb.OptimizationGoal.MAX # MIN/MAX
kind = cb.OptimizationKind.FINAL_VALUE # try AREA_UNDER_CURVE

data_pack = cb_configuration_series_api.best_n(n, field, uploaded_configurations, goal, kind)
plot_builder(data_pack).plot()

In [15]:
data_pack = cb_configuration_series_api.best_n_for_experiment(n, field, experiment_name, goal, kind)
plot_builder(data_pack).plot()

In [16]:
# Thresholds Configuration Series API

upper_threshold = 6.63
criteria = cb.CriteriaMode.ANY # ANY | FINAL | AVERAGE
thresholds = cb.Thresholds(upper=upper_threshold, lower=None)

data_pack = cb_configuration_series_api.exceeding_thresholds(field, criteria, thresholds, uploaded_configurations)
builder = plot_builder(data_pack)

# Additionaly draw threshold(s) on the plot
builder.add_trend('UPPER BOUND', cp.PlotlyTrendLine('average', cp.PlotlyTrendLineKind.CONST, upper_threshold))
builder.plot()

In [17]:
data_pack = cb_configuration_series_api.exceeding_thresholds_for_experiment(field, experiment_name, criteria, thresholds)
plot_builder(data_pack).plot()

In [28]:
# Cleanup
cb_deletion = cb.Deletion(cb_config)
cb_deletion.delete_computations(uploaded_computations)
cb_deletion.delete_configurations(uploaded_configurations)